# 1.2 - Transformación

El objetivo aquí será transformar las columnas del dataset `listings.csv`

In [1]:
# librerias

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

import re

In [2]:
listings=pd.read_csv('../data/clean_data/listings.csv')

for c in listings.select_dtypes(include='int'):
    listings[c]=pd.to_numeric(listings[c], downcast='integer')

listings.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21312 entries, 0 to 21311
Data columns (total 32 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            21312 non-null  int32  
 1   host_id                                       21312 non-null  int32  
 2   host_is_superhost                             21312 non-null  int8   
 3   neighbourhood_cleansed                        21312 non-null  object 
 4   neighbourhood_group_cleansed                  21312 non-null  object 
 5   latitude                                      21312 non-null  float64
 6   longitude                                     21312 non-null  float64
 7   property_type                                 21312 non-null  object 
 8   room_type                                     21312 non-null  object 
 9   accommodates                                  21312 non-null 

In [3]:
for c in listings.select_dtypes(include='float'):
    listings[c]=pd.to_numeric(listings[c], downcast='float')

listings.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21312 entries, 0 to 21311
Data columns (total 32 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            21312 non-null  int32  
 1   host_id                                       21312 non-null  int32  
 2   host_is_superhost                             21312 non-null  int8   
 3   neighbourhood_cleansed                        21312 non-null  object 
 4   neighbourhood_group_cleansed                  21312 non-null  object 
 5   latitude                                      21312 non-null  float32
 6   longitude                                     21312 non-null  float32
 7   property_type                                 21312 non-null  object 
 8   room_type                                     21312 non-null  object 
 9   accommodates                                  21312 non-null 

In [4]:
listings.head()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,6369,13660,1,Hispanoamérica,Chamartín,40.456280,-3.67763,Apartment,Private room,2,1,1,0,Real Bed,"{Wifi,""Air conditioning"",Kitchen,Elevator,Heat...",70,0,5,2,15,1,365,22,52,82,82,73,14,1,0,1,0
1,21853,83531,0,Cármenes,Latina,40.403412,-3.74084,Apartment,Private room,1,1,1,1,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",17,0,0,1,8,4,40,0,0,0,162,33,0,2,0,2,0
2,23001,82175,0,Legazpi,Arganzuela,40.386951,-3.69304,Apartment,Entire home/apt,6,2,3,5,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",""Wheelcha...",50,300,30,1,10,15,730,2,2,2,213,0,0,6,6,0,0
3,24805,101471,0,Universidad,Centro,40.422020,-3.70395,Apartment,Entire home/apt,3,1,0,1,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,E...",80,200,30,2,0,5,730,27,57,87,362,9,7,1,1,0,0
4,24836,101653,1,Justicia,Centro,40.419949,-3.69764,Apartment,Entire home/apt,4,1,2,3,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",115,200,0,4,0,3,10,24,54,77,342,67,15,1,1,0,0


**Columnas Numéricas**

Lo primero que haré será determinar que columnas numéricas voy a transformar y cómo.

In [5]:
listings.select_dtypes(np.number).columns

Index(['id', 'host_id', 'host_is_superhost', 'latitude', 'longitude',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms'],
      dtype='object')

+ id: id del airbnb. En realidad es categórico. No se toca.
+ host_id: id del host. En realidad es categórico. No se toca.
+ host_is_superhost: booleano (aunque sea 0-1). No se toca.
+ latitude: latitud del airbnb. 
+ longitude: longitud del airbnb. Ambas columnas se normalizan siguiendo las ecuaciones x=cos(lat) * cos(lon) ; y=cos(lat) * sin(lon) ; z=sin(lat) 
+ accommodates: nº de plazas. Normalizar.
+ bathrooms: nº de baños. Normalizar.
+ bedrooms: nº de habitaciones. Normalizar.
+ beds: nº de camas. Normalizar.
+ price: precio. Variable objetivo. En principio no lo voy a tocar, aunque más adelante probaré a predecir el log-precio, algo habitual en el mundo financiero.
+ security_deposit: fianza. Normalizar.
+ cleaning_fee: gastos de limpieza. Normalizar.
+ guests_included: nº inquilinos. Normalizar.
+ extra_people: dinero por persona extra. Normalizar.
+ minimum_nights: mínimo de noches. Normalizar.
+ maximum_nights: máximo de noches. Normalizar.
+ availability_30: disponibilidad 30 días. Normalizar.
+ availability_60: disponibilidad 60 días. Normalizar.
+ availability_90: disponibilidad 90 días. Normalizar.
+ availability_365: disponibilidad 365 días. Normalizar.
+ number_of_reviews: nº de reviews total. Normalizar.
+ number_of_reviews_ltm: nº de reviews últimos 12 meses. Normalizar.
+ calculated_host_listings_count: nº alquileres por host. Normalizar.
+ calculated_host_listings_count_entire_homes: nº alquileres enteros por host. Normalizar.
+ calculated_host_listings_count_private_rooms: nº alquileres hab. privada por host. Normalizar.
+ calculated_host_listings_count_shared_rooms: nº alquileres hab. compartida por host. Normalizar.

Probaré a normalizar de dos maneras distintas. Por un lado probaré la normalización a través del teorema central del límite (standardization) y también la transformación Box-Cox con distintas lambdas. El resumen de esta transformación es:

Valor Lambda (λ) | Data Transformada(Y’)
--------|------------
-3	| Y-3 = 1/Y3
-2	| Y-2 = 1/Y2
-1	| Y-1 = 1/Y1
-0.5	| Y-0.5 = 1/(√(Y))
0	| log(Y)**
0.5	| Y0.5 = √(Y)
1	| Y$^{1}$ = Y
2	| Y$^{2}$
3   | Y$^{3}$

In [6]:
# normalizaré al final, para probar distintos métodos

cols_to_norm=['accommodates', 'bathrooms', 'bedrooms', 'beds',
              'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
              'minimum_nights', 'maximum_nights', 'availability_30',
              'availability_60', 'availability_90', 'availability_365',
              'number_of_reviews', 'number_of_reviews_ltm',
              'calculated_host_listings_count',
              'calculated_host_listings_count_entire_homes',
              'calculated_host_listings_count_private_rooms',
              'calculated_host_listings_count_shared_rooms']

In [7]:
# x=cos(lat) * cos(lon) ; y=cos(lat) * sin(lon) ; z=sin(lat)
# de momento transformo lat-lon pero no las borro

listings['x']=np.cos(listings.latitude)*np.cos(listings.longitude)
listings['y']=np.cos(listings.latitude)*np.sin(listings.longitude)
listings['z']=np.sin(listings.latitude)

listings.head()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z
0,6369,13660,1,Hispanoamérica,Chamartín,40.456280,-3.67763,Apartment,Private room,2,1,1,0,Real Bed,"{Wifi,""Air conditioning"",Kitchen,Elevator,Heat...",70,0,5,2,15,1,365,22,52,82,82,73,14,1,0,1,0,0.796991,-0.473457,0.375026
1,21853,83531,0,Cármenes,Latina,40.403412,-3.74084,Apartment,Private room,1,1,1,1,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",17,0,0,1,8,4,40,0,0,0,162,33,0,2,0,2,0,0.748058,-0.510948,0.423488
2,23001,82175,0,Legazpi,Arganzuela,40.386951,-3.69304,Apartment,Entire home/apt,6,2,3,5,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",""Wheelcha...",50,300,30,1,10,15,730,2,2,2,213,0,0,6,6,0,0,0.765575,-0.470904,0.438342
3,24805,101471,0,Universidad,Centro,40.422020,-3.70395,Apartment,Entire home/apt,3,1,0,1,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,E...",80,200,30,2,0,5,730,27,57,87,362,9,7,1,1,0,0,0.772927,-0.487128,0.406559
4,24836,101653,1,Justicia,Centro,40.419949,-3.69764,Apartment,Entire home/apt,4,1,2,3,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",115,200,0,4,0,3,10,24,54,77,342,67,15,1,1,0,0,0.775268,-0.481796,0.408451


**Columnas Categóricas**

Veamos que transformaciones se harán para cada columna categórica:
    
+ neighbourhood_cleansed: esta columna se deja como esta.
+ neighbourhood_group_cleansed: esta también. La intención es hacer un join más adelante.
+ property_type: tipo de inmueble. Se hará One-Hot encoding (variables dummies)
+ room_type: tipo de alquiler. Se hará One-Hot encoding (variables dummies)
+ bed_type: tipo de cama. Se hará One-Hot encoding (variables dummies)
+ amenities: esta columna es la que más trabajo lleva. Trataré de hacer un One-Hot encoding con todos los amenities para cada uno de los pisos.

In [8]:
listings.select_dtypes('object').columns

Index(['neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'property_type', 'room_type', 'bed_type', 'amenities'],
      dtype='object')

In [9]:
one_hot_columns=['property_type', 'room_type', 'bed_type']

listings=pd.get_dummies(listings, columns=one_hot_columns, drop_first=True)

In [10]:
listings.head()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,property_type_Apartment,property_type_Barn,property_type_Bed and breakfast,property_type_Boutique hotel,property_type_Bungalow,property_type_Camper/RV,property_type_Casa particular (Cuba),property_type_Cave,property_type_Chalet,property_type_Condominium,property_type_Dome house,property_type_Earth house,property_type_Farm stay,property_type_Guest suite,property_type_Guesthouse,property_type_Hostel,property_type_Hotel,property_type_House,property_type_Hut,property_type_Igloo,property_type_Loft,property_type_Nature lodge,property_type_Other,property_type_Pension (South Korea),property_type_Serviced apartment,property_type_Tent,property_type_Tiny house,property_type_Townhouse,property_type_Villa,property_type_Yurt,room_type_Hotel room,room_type_Private room,room_type_Shared room,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed
0,6369,13660,1,Hispanoamérica,Chamartín,40.456280,-3.67763,2,1,1,0,"{Wifi,""Air conditioning"",Kitchen,Elevator,Heat...",70,0,5,2,15,1,365,22,52,82,82,73,14,1,0,1,0,0.796991,-0.473457,0.375026,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,21853,83531,0,Cármenes,Latina,40.403412,-3.74084,1,1,1,1,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",17,0,0,1,8,4,40,0,0,0,162,33,0,2,0,2,0,0.748058,-0.510948,0.423488,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,23001,82175,0,Legazpi,Arganzuela,40.386951,-3.69304,6,2,3,5,"{TV,Internet,Wifi,""Air conditioning"",""Wheelcha...",50,300,30,1,10,15,730,2,2,2,213,0,0,6,6,0,0,0.765575,-0.470904,0.438342,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,24805,101471,0,Universidad,Centro,40.422020,-3.70395,3,1,0,1,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,E...",80,200,30,2,0,5,730,27,57,87,362,9,7,1,1,0,0,0.772927,-0.487128,0.406559,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,24836,101653,1,Justicia,Centro,40.419949,-3.69764,4,1,2,3,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",115,200,0,4,0,3,10,24,54,77,342,67,15,1,1,0,0,0.775268,-0.481796,0.408451,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


**amenities**

In [11]:
amenities=listings.amenities.apply(lambda x: x.replace('{', '').replace('}', '').replace('"', ''))

In [12]:
all_amenities=set([e for fila in amenities for e in fila.split(',') if e!='' and 'translation missing' not in e])

len(all_amenities)

176

In [13]:
print(all_amenities)

{'Memory foam mattress', 'Beach essentials', 'Smart TV', 'Body soap', 'Day bed', 'Internet', 'Standing valet', 'TV', 'Dryer', ' toilet', 'Gas oven', 'Shower chair', 'Dog(s)', 'Safety card', 'Wine cooler', 'Disabled parking spot', 'Step-free shower', 'Wide entrance', 'Cooking basics', 'Formal dining area', 'HBO GO', 'Long term stays allowed', 'En suite bathroom', 'Sun loungers', 'Beach view', 'Pocket wifi', 'Netflix', 'Projector and screen', 'Oven', 'Balcony', 'Free street parking', 'Ethernet connection', 'Extra space around bed', 'Accessible-height toilet', 'Baby bath', 'Sound system', 'Patio or balcony', 'High chair', 'Fixed grab bars for toilet', 'First aid kit', 'Essentials', 'Toilet paper', 'Suitable for events', 'Wide entryway', 'EV charger', 'Mudroom', 'Coffee maker', 'Garden or backyard', 'Hot water', 'Family/kid friendly', 'Private bathroom', 'Paid parking on premises', 'Changing table', 'Handheld shower head', 'Single level home', 'Baby monitor', 'Wide clearance to shower', 'W

In [14]:
matrix_amenities=[[1 if e in fila else 0 for e in all_amenities] for fila in amenities]

In [15]:
listings.shape

(21312, 69)

In [16]:
len(matrix_amenities), len(matrix_amenities[0])

(21312, 176)

In [17]:
amenities_df=pd.DataFrame(matrix_amenities, columns=list(all_amenities))

In [18]:
amenities[0]

'Wifi,Air conditioning,Kitchen,Elevator,Heating,Family/kid friendly,Washer,Essentials,Shampoo,Hangers,Hair dryer,Iron,Hot water,Bed linens,Extra pillows and blankets,Pocket wifi'

In [19]:
amenities_df.head()

,Memory foam mattress,Beach essentials,Smart TV,Body soap,Day bed,Internet,Standing valet,TV,Dryer,toilet,Gas oven,Shower chair,Dog(s),Safety card,Wine cooler,Disabled parking spot,Step-free shower,Wide entrance,Cooking basics,Formal dining area,HBO GO,Long term stays allowed,En suite bathroom,Sun loungers,Beach view,Pocket wifi,Netflix,Projector and screen,Oven,Balcony,Free street parking,Ethernet connection,Extra space around bed,Accessible-height toilet,Baby bath,Sound system,Patio or balcony,High chair,Fixed grab bars for toilet,First aid kit,Essentials,Toilet paper,Suitable for events,Wide entryway,EV charger,Mudroom,Coffee maker,Garden or backyard,Hot water,Family/kid friendly,Private bathroom,Paid parking on premises,Changing table,Handheld shower head,Single level home,Baby monitor,Wide clearance to shower,Washer,Touchless faucets,Pets allowed,Crib,Other,Jetted tub,Steam oven,Host greets you,Trash can,Laptop friendly workspace,Private living room,Buzzer/wireless intercom,Refrigerator,Luggage dropoff allowed,Full kitchen,Smoke detector,Soaking tub,Heated floors,Mobile hoist,Indoor fireplace,Bedroom comforts,Central air conditioning,Hot tub,Cable TV,Heating,Washer / Dryer,Well-lit path to entrance,Pets live on this property,Iron,Bathroom essentials,High-resolution computer monitor,Hangers,Walk-in shower,Elevator,No stairs or steps to enter,Cat(s),Game console,Extra pillows and blankets,Ski-in/Ski-out,Room-darkening shades,Firm mattress,Fireplace guards,Lake access,Warming drawer,Kitchenette,Flat path to guest entrance,BBQ grill,Bidet,Air conditioning,Carbon monoxide detector,Breakfast table,DVD player,Cleaning before checkout,Window guards,Kitchen,Stair gates,Amazon Echo,Outlet covers,Wide hallways,Fixed grab bars for shower,Babysitter recommendations,Table corner guards,Pack ’n Play/travel crib,Gym,Wide entrance for guests,Children’s books and toys,Keypad,Outdoor seating,Printer,Rain shower,Waterfront,Hot water kettle,Wifi,Smart lock,Bath towel,Mini fridge,Convection oven,Terrace,Beachfront,Free parking on premises,Pillow-top mattress,Ceiling fan,Exercise equipment,Children’s dinnerware,Pool,Breakfast,Dishwasher,Bed linens,Self check-in,Hammock,Lock on bedroom door,Shampoo,Double oven,Barbecue utensils,Microwave,Other pet(s),Building staff,Bread maker,Lockbox,Shower gel,Fire extinguisher,Paid parking off premises,Stove,Hair dryer,24-hour check-in,Ground floor access,Doorman,Smoking allowed,Wheelchair accessible,Accessible-height bed,Dishes and silverware,Private entrance,Baking sheet,Bathtub,Wide doorway to guest bathroom,Heated towel rack,Shared pool,Espresso machine,Bathtub with bath chair
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0
3,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [20]:
amenities_df.describe()

,Memory foam mattress,Beach essentials,Smart TV,Body soap,Day bed,Internet,Standing valet,TV,Dryer,toilet,Gas oven,Shower chair,Dog(s),Safety card,Wine cooler,Disabled parking spot,Step-free shower,Wide entrance,Cooking basics,Formal dining area,HBO GO,Long term stays allowed,En suite bathroom,Sun loungers,Beach view,Pocket wifi,Netflix,Projector and screen,Oven,Balcony,Free street parking,Ethernet connection,Extra space around bed,Accessible-height toilet,Baby bath,Sound system,Patio or balcony,High chair,Fixed grab bars for toilet,First aid kit,Essentials,Toilet paper,Suitable for events,Wide entryway,EV charger,Mudroom,Coffee maker,Garden or backyard,Hot water,Family/kid friendly,Private bathroom,Paid parking on premises,Changing table,Handheld shower head,Single level home,Baby monitor,Wide clearance to shower,Washer,Touchless faucets,Pets allowed,Crib,Other,Jetted tub,Steam oven,Host greets you,Trash can,Laptop friendly workspace,Private living room,Buzzer/wireless intercom,Refrigerator,Luggage dropoff allowed,Full kitchen,Smoke detector,Soaking tub,Heated floors,Mobile hoist,Indoor fireplace,Bedroom comforts,Central air conditioning,Hot tub,Cable TV,Heating,Washer / Dryer,Well-lit path to entrance,Pets live on this property,Iron,Bathroom essentials,High-resolution computer monitor,Hangers,Walk-in shower,Elevator,No stairs or steps to enter,Cat(s),Game console,Extra pillows and blankets,Ski-in/Ski-out,Room-darkening shades,Firm mattress,Fireplace guards,Lake access,Warming drawer,Kitchenette,Flat path to guest entrance,BBQ grill,Bidet,Air conditioning,Carbon monoxide detector,Breakfast table,DVD player,Cleaning before checkout,Window guards,Kitchen,Stair gates,Amazon Echo,Outlet covers,Wide hallways,Fixed grab bars for shower,Babysitter recommendations,Table corner guards,Pack ’n Play/travel crib,Gym,Wide entrance for guests,Children’s books and toys,Keypad,Outdoor seating,Printer,Rain shower,Waterfront,Hot water kettle,Wifi,Smart lock,Bath towel,Mini fridge,Convection oven,Terrace,Beachfront,Free parking on premises,Pillow-top mattress,Ceiling fan,Exercise equipment,Children’s dinnerware,Pool,Breakfast,Dishwasher,Bed linens,Self check-in,Hammock,Lock on bedroom door,Shampoo,Double oven,Barbecue utensils,Microwave,Other pet(s),Building staff,Bread maker,Lockbox,Shower gel,Fire extinguisher,Paid parking off premises,Stove,Hair dryer,24-hour check-in,Ground floor access,Doorman,Smoking allowed,Wheelchair accessible,Accessible-height bed,Dishes and silverware,Private entrance,Baking sheet,Bathtub,Wide doorway to guest bathroom,Heated towel rack,Shared pool,Espresso machine,Bathtub with bath chair
count,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000000,21312.000

In [21]:
listings.drop('amenities', axis=1, inplace=True)

listings=pd.concat([listings, amenities_df], axis=1)

In [22]:
listings.head()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,property_type_Apartment,property_type_Barn,property_type_Bed and breakfast,property_type_Boutique hotel,property_type_Bungalow,property_type_Camper/RV,property_type_Casa particular (Cuba),property_type_Cave,property_type_Chalet,property_type_Condominium,property_type_Dome house,property_type_Earth house,property_type_Farm stay,property_type_Guest suite,property_type_Guesthouse,property_type_Hostel,property_type_Hotel,property_type_House,property_type_Hut,property_type_Igloo,property_type_Loft,property_type_Nature lodge,property_type_Other,property_type_Pension (South Korea),property_type_Serviced apartment,property_type_Tent,property_type_Tiny house,property_type_Townhouse,property_type_Villa,property_type_Yurt,room_type_Hotel room,room_type_Private room,room_type_Shared room,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,Memory foam mattress,Beach essentials,Smart TV,Body soap,Day bed,Internet,Standing valet,TV,Dryer,toilet,Gas oven,Shower chair,Dog(s),Safety card,Wine cooler,Disabled parking spot,Step-free shower,Wide entrance,Cooking basics,Formal dining area,HBO GO,Long term stays allowed,En suite bathroom,Sun loungers,Beach view,Pocket wifi,Netflix,Projector and screen,Oven,Balcony,Free street parking,Ethernet connection,Extra space around bed,Accessible-height toilet,Baby bath,Sound system,Patio or balcony,High chair,Fixed grab bars for toilet,First aid kit,Essentials,Toilet paper,Suitable for events,Wide entryway,EV charger,Mudroom,Coffee maker,Garden or backyard,Hot water,Family/kid friendly,Private bathroom,Paid parking on premises,Changing table,Handheld shower head,Single level home,Baby monitor,Wide clearance to shower,Washer,Touchless faucets,Pets allowed,Crib,Other,Jetted tub,Steam oven,Host greets you,Trash can,Laptop friendly workspace,Private living room,Buzzer/wireless intercom,Refrigerator,Luggage dropoff allowed,Full kitchen,Smoke detector,Soaking tub,Heated floors,Mobile hoist,Indoor fireplace,Bedroom comforts,Central air conditioning,Hot tub,Cable TV,Heating,Washer / Dryer,Well-lit path to entrance,Pets live on this property,Iron,Bathroom essentials,High-resolution computer monitor,Hangers,Walk-in shower,Elevator,No stairs or steps to enter,Cat(s),Game console,Extra pillows and blankets,Ski-in/Ski-out,Room-darkening shades,Firm mattress,Fireplace guards,Lake access,Warming drawer,Kitchenette,Flat path to guest entrance,BBQ grill,Bidet,Air conditioning,Carbon monoxide detector,Breakfast table,DVD player,Cleaning before checkout,Window guards,Kitchen,Stair gates,Amazon Echo,Outlet covers,Wide hallways,Fixed grab bars for shower,Babysitter recommendations,Table corner guards,Pack ’n Play/travel crib,Gym,Wide entrance for guests,Children’s books and toys,Keypad,Outdoor seating,Printer,Rain shower,Waterfront,Hot water kettle,Wifi,Smart lock,Bath towel,Mini fridge,Convection oven,Terrace,Beachfront,Free parking on premises,Pillow-top mattress,Ceiling fan,Exercise equipment,Children’s dinnerware,Pool,Breakfast,Dishwasher,Bed linens,Self check-in,Hammock,Lock on bedroom door,Shampoo,Double oven,Barbecue utensils,Microwave,Other pet(s),Building staff,Bread maker,Lockbox,Shower gel,Fire extinguisher,Paid parking off premises,Stove,Hair dryer,24-hour check-in,Ground floor access,Doorman,Smoking allowed,Wheelchair accessible,Accessible-height bed,Dishes and silverware,Private entrance,Baking sheet,Bathtub,Wide doorway to guest bathroom,Heated towel rack,Shared pool,Espresso machine,Bathtub with 

In [23]:
listings.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21312 entries, 0 to 21311
Columns: 244 entries, id to Bathtub with bath chair
dtypes: float32(5), int16(10), int32(3), int64(176), int8(11), object(2), uint8(37)
memory usage: 33.4 MB


In [24]:
listings.columns=[e.strip().replace(' ', '_').replace('/', '_').replace('-', '_').lower() for e in listings.columns]

listings=listings.rename(columns = lambda x: re.sub('[^A-Za-z0-9_]+', '', x))

In [25]:
listings.to_csv('../data/transform_data/listings.csv', index=False)

**Normalización**

In [26]:
listings_normal=listings.copy()

In [27]:
from sklearn.preprocessing import StandardScaler

In [28]:
scaler=StandardScaler()

In [29]:
listings_normal[cols_to_norm]=scaler.fit_transform(listings_normal[cols_to_norm])

listings_normal.head()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,property_type_apartment,property_type_barn,property_type_bed_and_breakfast,property_type_boutique_hotel,property_type_bungalow,property_type_camper_rv,property_type_casa_particular_cuba,property_type_cave,property_type_chalet,property_type_condominium,property_type_dome_house,property_type_earth_house,property_type_farm_stay,property_type_guest_suite,property_type_guesthouse,property_type_hostel,property_type_hotel,property_type_house,property_type_hut,property_type_igloo,property_type_loft,property_type_nature_lodge,property_type_other,property_type_pension_south_korea,property_type_serviced_apartment,property_type_tent,property_type_tiny_house,property_type_townhouse,property_type_villa,property_type_yurt,room_type_hotel_room,room_type_private_room,room_type_shared_room,bed_type_couch,bed_type_futon,bed_type_pull_out_sofa,bed_type_real_bed,memory_foam_mattress,beach_essentials,smart_tv,body_soap,day_bed,internet,standing_valet,tv,dryer,toilet,gas_oven,shower_chair,dogs,safety_card,wine_cooler,disabled_parking_spot,step_free_shower,wide_entrance,cooking_basics,formal_dining_area,hbo_go,long_term_stays_allowed,en_suite_bathroom,sun_loungers,beach_view,pocket_wifi,netflix,projector_and_screen,oven,balcony,free_street_parking,ethernet_connection,extra_space_around_bed,accessible_height_toilet,baby_bath,sound_system,patio_or_balcony,high_chair,fixed_grab_bars_for_toilet,first_aid_kit,essentials,toilet_paper,suitable_for_events,wide_entryway,ev_charger,mudroom,coffee_maker,garden_or_backyard,hot_water,family_kid_friendly,private_bathroom,paid_parking_on_premises,changing_table,handheld_shower_head,single_level_home,baby_monitor,wide_clearance_to_shower,washer,touchless_faucets,pets_allowed,crib,other,jetted_tub,steam_oven,host_greets_you,trash_can,laptop_friendly_workspace,private_living_room,buzzer_wireless_intercom,refrigerator,luggage_dropoff_allowed,full_kitchen,smoke_detector,soaking_tub,heated_floors,mobile_hoist,indoor_fireplace,bedroom_comforts,central_air_conditioning,hot_tub,cable_tv,heating,washer___dryer,well_lit_path_to_entrance,pets_live_on_this_property,iron,bathroom_essentials,high_resolution_computer_monitor,hangers,walk_in_shower,elevator,no_stairs_or_steps_to_enter,cats,game_console,extra_pillows_and_blankets,ski_in_ski_out,room_darkening_shades,firm_mattress,fireplace_guards,lake_access,warming_drawer,kitchenette,flat_path_to_guest_entrance,bbq_grill,bidet,air_conditioning,carbon_monoxide_detector,breakfast_table,dvd_player,cleaning_before_checkout,window_guards,kitchen,stair_gates,amazon_echo,outlet_covers,wide_hallways,fixed_grab_bars_for_shower,babysitter_recommendations,table_corner_guards,pack_n_play_travel_crib,gym,wide_entrance_for_guests,childrens_books_and_toys,keypad,outdoor_seating,printer,rain_shower,waterfront,hot_water_kettle,wifi,smart_lock,bath_towel,mini_fridge,convection_oven,terrace,beachfront,free_parking_on_premises,pillow_top_mattress,ceiling_fan,exercise_equipment,childrens_dinnerware,pool,breakfast,dishwasher,bed_linens,self_check_in,hammock,lock_on_bedroom_door,shampoo,double_oven,barbecue_utensils,microwave,other_pets,building_staff,bread_maker,lockbox,shower_gel,fire_extinguisher,paid_parking_off_premises,stove,hair_dryer,24_hour_check_in,ground_floor_access,doorman,smoking_allowed,wheelchair_accessible,accessible_height_bed,dishes_and_silverware,private_entrance,baking_sheet,bathtub,wide_doorway_to_guest_bathroom,heated_towel_rack,shared_pool,espresso_machine,bathtub_with_bath_chair
0,

In [30]:
listings_normal.to_csv('../data/transform_data/listings_normal.csv', index=False)

In [31]:
from scipy.stats import boxcox

In [32]:
listings_box06=listings.copy()

In [33]:
listings_box06[cols_to_norm]=boxcox(listings_box06[cols_to_norm], lmbda=0.618033)

listings_box06.head()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,property_type_apartment,property_type_barn,property_type_bed_and_breakfast,property_type_boutique_hotel,property_type_bungalow,property_type_camper_rv,property_type_casa_particular_cuba,property_type_cave,property_type_chalet,property_type_condominium,property_type_dome_house,property_type_earth_house,property_type_farm_stay,property_type_guest_suite,property_type_guesthouse,property_type_hostel,property_type_hotel,property_type_house,property_type_hut,property_type_igloo,property_type_loft,property_type_nature_lodge,property_type_other,property_type_pension_south_korea,property_type_serviced_apartment,property_type_tent,property_type_tiny_house,property_type_townhouse,property_type_villa,property_type_yurt,room_type_hotel_room,room_type_private_room,room_type_shared_room,bed_type_couch,bed_type_futon,bed_type_pull_out_sofa,bed_type_real_bed,memory_foam_mattress,beach_essentials,smart_tv,body_soap,day_bed,internet,standing_valet,tv,dryer,toilet,gas_oven,shower_chair,dogs,safety_card,wine_cooler,disabled_parking_spot,step_free_shower,wide_entrance,cooking_basics,formal_dining_area,hbo_go,long_term_stays_allowed,en_suite_bathroom,sun_loungers,beach_view,pocket_wifi,netflix,projector_and_screen,oven,balcony,free_street_parking,ethernet_connection,extra_space_around_bed,accessible_height_toilet,baby_bath,sound_system,patio_or_balcony,high_chair,fixed_grab_bars_for_toilet,first_aid_kit,essentials,toilet_paper,suitable_for_events,wide_entryway,ev_charger,mudroom,coffee_maker,garden_or_backyard,hot_water,family_kid_friendly,private_bathroom,paid_parking_on_premises,changing_table,handheld_shower_head,single_level_home,baby_monitor,wide_clearance_to_shower,washer,touchless_faucets,pets_allowed,crib,other,jetted_tub,steam_oven,host_greets_you,trash_can,laptop_friendly_workspace,private_living_room,buzzer_wireless_intercom,refrigerator,luggage_dropoff_allowed,full_kitchen,smoke_detector,soaking_tub,heated_floors,mobile_hoist,indoor_fireplace,bedroom_comforts,central_air_conditioning,hot_tub,cable_tv,heating,washer___dryer,well_lit_path_to_entrance,pets_live_on_this_property,iron,bathroom_essentials,high_resolution_computer_monitor,hangers,walk_in_shower,elevator,no_stairs_or_steps_to_enter,cats,game_console,extra_pillows_and_blankets,ski_in_ski_out,room_darkening_shades,firm_mattress,fireplace_guards,lake_access,warming_drawer,kitchenette,flat_path_to_guest_entrance,bbq_grill,bidet,air_conditioning,carbon_monoxide_detector,breakfast_table,dvd_player,cleaning_before_checkout,window_guards,kitchen,stair_gates,amazon_echo,outlet_covers,wide_hallways,fixed_grab_bars_for_shower,babysitter_recommendations,table_corner_guards,pack_n_play_travel_crib,gym,wide_entrance_for_guests,childrens_books_and_toys,keypad,outdoor_seating,printer,rain_shower,waterfront,hot_water_kettle,wifi,smart_lock,bath_towel,mini_fridge,convection_oven,terrace,beachfront,free_parking_on_premises,pillow_top_mattress,ceiling_fan,exercise_equipment,childrens_dinnerware,pool,breakfast,dishwasher,bed_linens,self_check_in,hammock,lock_on_bedroom_door,shampoo,double_oven,barbecue_utensils,microwave,other_pets,building_staff,bread_maker,lockbox,shower_gel,fire_extinguisher,paid_parking_off_premises,stove,hair_dryer,24_hour_check_in,ground_floor_access,doorman,smoking_allowed,wheelchair_accessible,accessible_height_bed,dishes_and_silverware,private_entrance,baking_sheet,bathtub,wide_doorway_to_guest_bathroom,heated_towel_rack,shared_pool,espresso_machine,bathtub_with_bath_chair
0,

In [34]:
listings_box06.to_csv('../data/transform_data/listings_box06.csv', index=False)

In [35]:
listings_box16=listings.copy()

In [36]:
listings_box16[cols_to_norm]=boxcox(listings_box16[cols_to_norm], lmbda=1.618033)

listings_box16.head()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,property_type_apartment,property_type_barn,property_type_bed_and_breakfast,property_type_boutique_hotel,property_type_bungalow,property_type_camper_rv,property_type_casa_particular_cuba,property_type_cave,property_type_chalet,property_type_condominium,property_type_dome_house,property_type_earth_house,property_type_farm_stay,property_type_guest_suite,property_type_guesthouse,property_type_hostel,property_type_hotel,property_type_house,property_type_hut,property_type_igloo,property_type_loft,property_type_nature_lodge,property_type_other,property_type_pension_south_korea,property_type_serviced_apartment,property_type_tent,property_type_tiny_house,property_type_townhouse,property_type_villa,property_type_yurt,room_type_hotel_room,room_type_private_room,room_type_shared_room,bed_type_couch,bed_type_futon,bed_type_pull_out_sofa,bed_type_real_bed,memory_foam_mattress,beach_essentials,smart_tv,body_soap,day_bed,internet,standing_valet,tv,dryer,toilet,gas_oven,shower_chair,dogs,safety_card,wine_cooler,disabled_parking_spot,step_free_shower,wide_entrance,cooking_basics,formal_dining_area,hbo_go,long_term_stays_allowed,en_suite_bathroom,sun_loungers,beach_view,pocket_wifi,netflix,projector_and_screen,oven,balcony,free_street_parking,ethernet_connection,extra_space_around_bed,accessible_height_toilet,baby_bath,sound_system,patio_or_balcony,high_chair,fixed_grab_bars_for_toilet,first_aid_kit,essentials,toilet_paper,suitable_for_events,wide_entryway,ev_charger,mudroom,coffee_maker,garden_or_backyard,hot_water,family_kid_friendly,private_bathroom,paid_parking_on_premises,changing_table,handheld_shower_head,single_level_home,baby_monitor,wide_clearance_to_shower,washer,touchless_faucets,pets_allowed,crib,other,jetted_tub,steam_oven,host_greets_you,trash_can,laptop_friendly_workspace,private_living_room,buzzer_wireless_intercom,refrigerator,luggage_dropoff_allowed,full_kitchen,smoke_detector,soaking_tub,heated_floors,mobile_hoist,indoor_fireplace,bedroom_comforts,central_air_conditioning,hot_tub,cable_tv,heating,washer___dryer,well_lit_path_to_entrance,pets_live_on_this_property,iron,bathroom_essentials,high_resolution_computer_monitor,hangers,walk_in_shower,elevator,no_stairs_or_steps_to_enter,cats,game_console,extra_pillows_and_blankets,ski_in_ski_out,room_darkening_shades,firm_mattress,fireplace_guards,lake_access,warming_drawer,kitchenette,flat_path_to_guest_entrance,bbq_grill,bidet,air_conditioning,carbon_monoxide_detector,breakfast_table,dvd_player,cleaning_before_checkout,window_guards,kitchen,stair_gates,amazon_echo,outlet_covers,wide_hallways,fixed_grab_bars_for_shower,babysitter_recommendations,table_corner_guards,pack_n_play_travel_crib,gym,wide_entrance_for_guests,childrens_books_and_toys,keypad,outdoor_seating,printer,rain_shower,waterfront,hot_water_kettle,wifi,smart_lock,bath_towel,mini_fridge,convection_oven,terrace,beachfront,free_parking_on_premises,pillow_top_mattress,ceiling_fan,exercise_equipment,childrens_dinnerware,pool,breakfast,dishwasher,bed_linens,self_check_in,hammock,lock_on_bedroom_door,shampoo,double_oven,barbecue_utensils,microwave,other_pets,building_staff,bread_maker,lockbox,shower_gel,fire_extinguisher,paid_parking_off_premises,stove,hair_dryer,24_hour_check_in,ground_floor_access,doorman,smoking_allowed,wheelchair_accessible,accessible_height_bed,dishes_and_silverware,private_entrance,baking_sheet,bathtub,wide_doorway_to_guest_bathroom,heated_towel_rack,shared_pool,espresso_machine,bathtub_with_bath_chair
0,

In [37]:
listings_box16.to_csv('../data/transform_data/listings_box16.csv', index=False)

In [38]:
listings_box26=listings.copy()

In [39]:
listings_box26[cols_to_norm]=boxcox(listings_box26[cols_to_norm], lmbda=2.618033)

listings_box26.head()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,property_type_apartment,property_type_barn,property_type_bed_and_breakfast,property_type_boutique_hotel,property_type_bungalow,property_type_camper_rv,property_type_casa_particular_cuba,property_type_cave,property_type_chalet,property_type_condominium,property_type_dome_house,property_type_earth_house,property_type_farm_stay,property_type_guest_suite,property_type_guesthouse,property_type_hostel,property_type_hotel,property_type_house,property_type_hut,property_type_igloo,property_type_loft,property_type_nature_lodge,property_type_other,property_type_pension_south_korea,property_type_serviced_apartment,property_type_tent,property_type_tiny_house,property_type_townhouse,property_type_villa,property_type_yurt,room_type_hotel_room,room_type_private_room,room_type_shared_room,bed_type_couch,bed_type_futon,bed_type_pull_out_sofa,bed_type_real_bed,memory_foam_mattress,beach_essentials,smart_tv,body_soap,day_bed,internet,standing_valet,tv,dryer,toilet,gas_oven,shower_chair,dogs,safety_card,wine_cooler,disabled_parking_spot,step_free_shower,wide_entrance,cooking_basics,formal_dining_area,hbo_go,long_term_stays_allowed,en_suite_bathroom,sun_loungers,beach_view,pocket_wifi,netflix,projector_and_screen,oven,balcony,free_street_parking,ethernet_connection,extra_space_around_bed,accessible_height_toilet,baby_bath,sound_system,patio_or_balcony,high_chair,fixed_grab_bars_for_toilet,first_aid_kit,essentials,toilet_paper,suitable_for_events,wide_entryway,ev_charger,mudroom,coffee_maker,garden_or_backyard,hot_water,family_kid_friendly,private_bathroom,paid_parking_on_premises,changing_table,handheld_shower_head,single_level_home,baby_monitor,wide_clearance_to_shower,washer,touchless_faucets,pets_allowed,crib,other,jetted_tub,steam_oven,host_greets_you,trash_can,laptop_friendly_workspace,private_living_room,buzzer_wireless_intercom,refrigerator,luggage_dropoff_allowed,full_kitchen,smoke_detector,soaking_tub,heated_floors,mobile_hoist,indoor_fireplace,bedroom_comforts,central_air_conditioning,hot_tub,cable_tv,heating,washer___dryer,well_lit_path_to_entrance,pets_live_on_this_property,iron,bathroom_essentials,high_resolution_computer_monitor,hangers,walk_in_shower,elevator,no_stairs_or_steps_to_enter,cats,game_console,extra_pillows_and_blankets,ski_in_ski_out,room_darkening_shades,firm_mattress,fireplace_guards,lake_access,warming_drawer,kitchenette,flat_path_to_guest_entrance,bbq_grill,bidet,air_conditioning,carbon_monoxide_detector,breakfast_table,dvd_player,cleaning_before_checkout,window_guards,kitchen,stair_gates,amazon_echo,outlet_covers,wide_hallways,fixed_grab_bars_for_shower,babysitter_recommendations,table_corner_guards,pack_n_play_travel_crib,gym,wide_entrance_for_guests,childrens_books_and_toys,keypad,outdoor_seating,printer,rain_shower,waterfront,hot_water_kettle,wifi,smart_lock,bath_towel,mini_fridge,convection_oven,terrace,beachfront,free_parking_on_premises,pillow_top_mattress,ceiling_fan,exercise_equipment,childrens_dinnerware,pool,breakfast,dishwasher,bed_linens,self_check_in,hammock,lock_on_bedroom_door,shampoo,double_oven,barbecue_utensils,microwave,other_pets,building_staff,bread_maker,lockbox,shower_gel,fire_extinguisher,paid_parking_off_premises,stove,hair_dryer,24_hour_check_in,ground_floor_access,doorman,smoking_allowed,wheelchair_accessible,accessible_height_bed,dishes_and_silverware,private_entrance,baking_sheet,bathtub,wide_doorway_to_guest_bathroom,heated_towel_rack,shared_pool,espresso_machine,bathtub_with_bath_chair
0,

In [40]:
listings_box26.to_csv('../data/transform_data/listings_box26.csv', index=False)